# Simple RNN — Prédiction autorégressiive et Vanishing Gradient

## Objectif
Ce notebook construit un **SimpleRNN** pour prédire `Total_reservations`,
tout en illustrant le **problème du vanishing gradient**.

## Formule du SimpleRNN
$$h_t = \tanh(W_{ih}\, x_t + b_{ih} + W_{hh}\, h_{t-1} + b_{hh})$$

## Pourquoi le vanishing gradient ?
Lors de la BPTT (BackPropagation Through Time), le gradient doit traverser
le produit de matrices récurrentes :
$$\frac{\partial \mathcal{L}}{\partial h_1} = \frac{\partial \mathcal{L}}{\partial h_T}
\prod_{t=2}^{T} W_{hh}^\top \mathrm{diag}(\tanh'(h_{t-1}))$$
- $\tanh'(x) \leq 1$ → chaque facteur atténue le gradient
- Si $\|W_{hh}\| < 1$ → le gradient **disparaît exponentiellement** en $T$
- Le réseau oublie les dépendances éloignées → LSTM/GRU nécessaires

## Paramètres comparables inter-modèles
| Paramètre | Valeur | Rôle |
|-----------|--------|------|
| `WINDOW` | 5 | Jours de contexte (1 semaine ouvrée) |
| `N_TEST` | 5 | Jours à prédire (dernière semaine) |
| `TARGET_COL` | `Total_reservations` | Variable cible |
| `EPOCHS` | 200 | Epochs max d'entraînement |
| `HIDDEN_SIZE` | 64 | Dimension de l'état caché |

## 1 — Configuration globale

**Modifiez uniquement ce bloc** pour changer les hyperparamètres.

In [ ]:
# ─────────────────────────────────────────────────────────────────
# PARAMÈTRES GLOBAUX — modifier ici pour ajuster l'expérience
# ─────────────────────────────────────────────────────────────────
DATA_PATH   = '../data/df_venues_final.csv'
TARGET_COL  = 'GLOBAL'
MODEL_NAME  = 'RNN'

WINDOW      = 5      # jours de contexte (fenêtre glissante)
N_TEST      = 5      # jours de test = dernière semaine

# Entraînement
EPOCHS      = 200
LR          = 1e-3
BATCH_SIZE  = 16
WEIGHT_DECAY = 1e-5
PATIENCE    = 20

# Architecture RNN
HIDDEN_SIZE = 64
NUM_LAYERS  = 1     # SimpleRNN : 1 couche (déjà complexe pour vanishing gradient)
DROPOUT     = 0.0   # pas de dropout sur 1 couche

# Expérience vanishing gradient : tailles de fenêtre à tester
WINDOWS_EXP     = [2, 5, 10, 20]
EPOCHS_EXP      = 50

FEATURES = [
    'GLOBAL',
    'Total_reservations', 'Temp', 'pluie', 'autre',
    'Greve_nationale', 'prof_nationale',
    'jour_ferie.', 'pont.conge.', 'holiday',
    'jour_lundi', 'jour_mardi', 'jour_mercredi', 'jour_jeudi', 'jour_vendredi',
]
TARGET_IDX = 0

DAY_LABELS = ['Lundi', 'Mardi', 'Mercredi', 'Jeudi', 'Vendredi']
IMG_PATH   = f'../img/{MODEL_NAME.lower()}_results.png'

## 2 — Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score

torch.manual_seed(42)
np.random.seed(42)

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'PyTorch : {torch.__version__} | Device : {DEVICE}')

## 3 — Chargement et tri chronologique

In [ ]:
df = pd.read_csv(DATA_PATH, sep=';')

day_order = {
    'jour_lundi': 0, 'jour_mardi': 1, 'jour_mercredi': 2,
    'jour_jeudi': 3, 'jour_vendredi': 4,
}
for col in day_order:
    df[col] = df[col].map({'True': 1, 'False': 0}).fillna(df[col]).astype(int)

df['day_num'] = df[list(day_order.keys())].idxmax(axis=1).map(day_order)
df = df.sort_values(['Annee', 'Semaine', 'day_num']).reset_index(drop=True)

print(f'Shape : {df.shape}')
_last = df.iloc[-N_TEST:]
TEST_LABEL = f"S{int(_last.iloc[0]['Semaine'])}/{int(_last.iloc[0]['Annee'])}"
print(f'Période : {df["Annee"].min()}/S{df["Semaine"].min()} → {df["Annee"].max()}/S{df["Semaine"].max()}')
print(f'\n--- Dernière semaine de test ({TEST_LABEL}) ---')
print(df[['Annee', 'Semaine', 'day_num', TARGET_COL]].tail(N_TEST).to_string())

## 4 — Prétraitement et séquences

In [ ]:
scaler   = MinMaxScaler()
X_scaled = scaler.fit_transform(df[FEATURES].astype(float))

X_train_raw = X_scaled[:-N_TEST]
X_test_raw  = X_scaled[-N_TEST:]

def make_sequences(X, window):
    Xs, ys = [], []
    for i in range(window, len(X)):
        Xs.append(X[i - window : i])
        ys.append(X[i, TARGET_IDX])
    return np.array(Xs, dtype=np.float32), np.array(ys, dtype=np.float32).reshape(-1, 1)

X_seq, y_seq = make_sequences(X_train_raw, WINDOW)

print(f'Séquences train : {X_seq.shape}  →  {y_seq.shape}')

X_tr = torch.tensor(X_seq).to(DEVICE)
y_tr = torch.tensor(y_seq).to(DEVICE)
train_loader = DataLoader(TensorDataset(X_tr, y_tr), batch_size=BATCH_SIZE, shuffle=True)

## 5 — Architecture SimpleRNN

```
Input  : (batch, WINDOW=5, n_features=15)
   ↓
RNN(hidden=64, layers=1, nonlinearity='tanh')
   ↓  h_T  (batch, 64)
Linear(64 → 1)
   ↓
Total_reservations normalisé [0, 1]
```

**Différence vs LSTM/GRU :** aucune porte de mémoire → le gradient disparaît sur de longues séquences.
On capture les gradients de `W_hh` (poids récurrents) pour visualiser ce phénomène.

In [ ]:
class SimpleRNNModel(nn.Module):
    def __init__(self, input_size, hidden_size=HIDDEN_SIZE,
                 num_layers=NUM_LAYERS):
        super().__init__()
        self.rnn = nn.RNN(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            nonlinearity='tanh',
        )
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        out, _ = self.rnn(x)
        return self.fc(out[:, -1, :])


model = SimpleRNNModel(input_size=len(FEATURES)).to(DEVICE)
print(model)
print(f'\nNombre de paramètres : {sum(p.numel() for p in model.parameters()):,}')
print(f'(vs LSTM ≈ 54k, GRU ≈ 40k)')

## 6 — Entraînement + capture des gradients

On enregistre la **norme L2 du gradient** de `W_hh` (poids récurrents) à chaque epoch.
C'est cette matrice qui est répétitivement multipliée lors de la BPTT → source du vanishing gradient.

In [ ]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', patience=PATIENCE, factor=0.5
)

loss_history = []
grad_history = {'rnn.weight_ih_l0': [], 'rnn.weight_hh_l0': [], 'fc.weight': []}

for epoch in range(1, EPOCHS + 1):
    model.train()
    epoch_loss = 0.0
    for xb, yb in train_loader:
        optimizer.zero_grad()
        loss = criterion(model(xb), yb)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        epoch_loss += loss.item() * xb.size(0)
    epoch_loss /= len(X_seq)
    loss_history.append(epoch_loss)
    scheduler.step(epoch_loss)

    for name, param in model.named_parameters():
        if name in grad_history and param.grad is not None:
            grad_history[name].append(param.grad.norm().item())

    if epoch % 50 == 0:
        lr_cur = optimizer.param_groups[0]['lr']
        whh_grad = grad_history['rnn.weight_hh_l0'][-1] if grad_history['rnn.weight_hh_l0'] else 0
        print(f'Epoch {epoch:3d}/{EPOCHS} | Loss : {epoch_loss:.5f} | '
              f'LR : {lr_cur:.6f} | ‖∇W_hh‖ : {whh_grad:.6f}')

print('\nEntraînement terminé.')

## 7 — Prédiction autorégressiive (fenêtre glissante)

In [ ]:
model.eval()

seed_window = X_scaled[-N_TEST - WINDOW : -N_TEST].copy()
preds_norm  = []
window      = seed_window.copy()

print('=== Prédiction autorégressiive ===')
print(f"{'Étape':<6} {'Jour':<10} {'ŷ norm':>8}   Fenêtre {TARGET_COL} [normalisé]")
print('-' * 70)

for step in range(N_TEST):
    x = torch.tensor(window, dtype=torch.float32).unsqueeze(0).to(DEVICE)
    with torch.no_grad():
        pred_n = model(x).item()
    preds_norm.append(pred_n)

    print(f'  {step}     {DAY_LABELS[step]:<10} {pred_n:>8.4f}   '
          f'{np.round(window[:, TARGET_IDX], 3)}')

    new_day             = X_test_raw[step].copy()
    new_day[TARGET_IDX] = pred_n
    window = np.vstack([window[1:], new_day.reshape(1, -1)])

dummy                = np.zeros((N_TEST, len(FEATURES)))
dummy[:, TARGET_IDX] = preds_norm
preds_real = scaler.inverse_transform(dummy)[:, TARGET_IDX]
real_real  = df[TARGET_COL].values[-N_TEST:]

print('\n=== Résultats ===')
print(f"{'Jour':<10} | {'Réel':>6} | {MODEL_NAME:>6} | {'Erreur':>7} | {'Erreur %':>8}")
print('-' * 48)
for i, d in enumerate(DAY_LABELS):
    err    = preds_real[i] - real_real[i]
    err_pc = 100 * abs(err) / real_real[i]
    print(f'{d:<10} | {real_real[i]:>6.0f} | {preds_real[i]:>6.0f} | {err:>+7.0f} | {err_pc:>7.1f}%')

## 8 — Métriques

In [ ]:
mae  = mean_absolute_error(real_real, preds_real)
mape = mean_absolute_percentage_error(real_real, preds_real) * 100
rmse = np.sqrt(np.mean((real_real - preds_real) ** 2))
r2   = r2_score(real_real, preds_real)

print(f'╔{"═" * 50}╗')
print(f'║  Métriques {MODEL_NAME} — {TEST_LABEL:<{37}}║')
print(f'╠{"═" * 50}╣')
print(f'║  MAE   (Erreur absolue moyenne)  : {mae:>7.1f} réserv. ║')
print(f'║  MAPE  (Erreur % moyenne)        : {mape:>6.2f}%          ║')
print(f'║  RMSE  (Racine erreur quadrat.)  : {rmse:>7.1f} réserv. ║')
print(f'║  R²    (Coefficient det.)        : {r2:>7.4f}           ║')
print(f'╚{"═" * 50}╝')

## 9 — Visualisations standardisées

Identiques aux autres modèles + un graphique supplémentaire sur les gradients.

In [ ]:
all_real     = df[TARGET_COL].values
test_indices = list(range(len(all_real) - N_TEST, len(all_real)))

fig, axes = plt.subplots(1, 3, figsize=(18, 5))
fig.suptitle(f'{MODEL_NAME} — Prédiction autorégressiive ({TEST_LABEL})',
             fontsize=14, fontweight='bold')

# ── 1 : Courbe d'apprentissage ────────────────────────────────────────────
ax = axes[0]
ax.plot(loss_history, color='mediumpurple', lw=1.5)
ax.set_xlabel('Epoch')
ax.set_ylabel('MSE (normalisé)')
ax.set_title('Courbe d\'apprentissage')
ax.grid(alpha=0.3)
ax.annotate(f'Loss finale : {loss_history[-1]:.5f}',
            xy=(len(loss_history) - 1, loss_history[-1]),
            xytext=(-80, 20), textcoords='offset points',
            arrowprops=dict(arrowstyle='->', color='gray'), fontsize=9)

# ── 2 : Série complète + prédiction ─────────────────────────────────────
ax = axes[1]
ax.plot(range(len(all_real)), all_real,
        label='Réservations réelles', color='steelblue', alpha=0.7, lw=1.2)
ax.plot(test_indices, preds_real,
        label=f'{MODEL_NAME} autorégressif', color='mediumpurple',
        ls='--', marker='o', ms=7, lw=2)
ax.plot(test_indices, real_real,
        color='steelblue', marker='x', ms=9, ls='None', label=f'Réel ({TEST_LABEL})')
ax.axvline(x=test_indices[0] - 0.5, color='gray', ls=':', lw=1.5, label='Début test')
ax.set_xlabel('Jours (ordre chronologique)')
ax.set_ylabel(TARGET_COL)
ax.set_title(f'Prédiction — fenêtre={WINDOW} jours')
ax.legend(fontsize=8)
ax.grid(alpha=0.3)

axins = ax.inset_axes([0.60, 0.05, 0.38, 0.45])
axins.plot(DAY_LABELS, real_real, 'o-', color='steelblue', ms=5)
axins.plot(DAY_LABELS, preds_real, 's--', color='mediumpurple', ms=5)
axins.set_title(f'Zoom {TEST_LABEL}', fontsize=8)
axins.tick_params(axis='x', labelsize=6)
axins.grid(alpha=0.3)

# ── 3 : Barres côte à côte ───────────────────────────────────────────────
ax    = axes[2]
x     = np.arange(N_TEST)
width = 0.35
b_r   = ax.bar(x - width/2, real_real,  width, label='Réel',      color='steelblue',   alpha=0.85)
b_p   = ax.bar(x + width/2, preds_real, width, label=MODEL_NAME,  color='mediumpurple', alpha=0.85)
for bar, val in zip(b_r, real_real):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 2,
            f'{val:.0f}', ha='center', va='bottom', fontsize=8, color='steelblue')
for bar, val in zip(b_p, preds_real):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 2,
            f'{val:.0f}', ha='center', va='bottom', fontsize=8, color='mediumpurple')
ax.set_xticks(x)
ax.set_xticklabels(DAY_LABELS)
ax.set_ylabel(TARGET_COL)
ax.set_title('Comparaison jour par jour')
ax.legend()
ax.grid(axis='y', alpha=0.3)
for i in range(N_TEST):
    err = preds_real[i] - real_real[i]
    ax.text(i, max(real_real[i], preds_real[i]) + 12,
            f'{err:+.0f}', ha='center', fontsize=7, color='darkred')

plt.tight_layout()
plt.savefig(IMG_PATH, dpi=150, bbox_inches='tight')
plt.show()
print(f'Figure sauvegardée → {IMG_PATH}')

## 10 — Analyse du Vanishing Gradient

### Visualisation des normes de gradients par couche
On compare :
- `W_ih` : poids entrée → caché (gradient de surface)
- `W_hh` : **poids récurrents** (source du vanishing)
- `W_fc` : poids de la couche de sortie (gradient fort, proche de la loss)

On s'attend à voir `W_hh` avec une norme bien plus faible que `W_fc`.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
fig.suptitle('Analyse du Vanishing Gradient — SimpleRNN', fontsize=13, fontweight='bold')

# ── Normes de gradients par couche ────────────────────────────────────────
ax = axes[0]
colors = {'rnn.weight_ih_l0': 'steelblue', 'rnn.weight_hh_l0': 'tomato', 'fc.weight': 'green'}
labels = {'rnn.weight_ih_l0': 'W_ih  (entrée → caché)',
          'rnn.weight_hh_l0': 'W_hh  (récurrent — vanishing !)',
          'fc.weight'        : 'W_fc  (couche de sortie)'}
for name, grads in grad_history.items():
    if grads:
        ax.plot(grads, label=labels[name], color=colors[name], lw=1.5)
ax.set_xlabel('Epoch')
ax.set_ylabel('‖∇W‖₂')
ax.set_title('Norme des gradients par couche')
ax.legend(fontsize=9)
ax.grid(alpha=0.3)
ax.set_yscale('log')

# ── Vanishing gradient selon la longueur de fenêtre ───────────────────────
ax = axes[1]

results_vg = {}
for win in WINDOWS_EXP:
    Xs, ys = make_sequences(X_scaled[:-N_TEST], win)
    Xt = torch.tensor(Xs, dtype=torch.float32).to(DEVICE)
    yt = torch.tensor(ys, dtype=torch.float32).to(DEVICE)
    loader_w = DataLoader(TensorDataset(Xt, yt), batch_size=BATCH_SIZE, shuffle=True)

    m = SimpleRNNModel(len(FEATURES), HIDDEN_SIZE, 1).to(DEVICE)
    opt_w = torch.optim.Adam(m.parameters(), lr=LR)
    crit  = nn.MSELoss()
    grad_norms_hh = []

    for ep in range(EPOCHS_EXP):
        m.train()
        for xb, yb in loader_w:
            opt_w.zero_grad()
            loss = crit(m(xb), yb)
            loss.backward()
            opt_w.step()
        if m.rnn.weight_hh_l0.grad is not None:
            grad_norms_hh.append(m.rnn.weight_hh_l0.grad.norm().item())
    results_vg[win] = grad_norms_hh

palette = plt.cm.viridis(np.linspace(0.2, 0.9, len(WINDOWS_EXP)))
for color, win in zip(palette, WINDOWS_EXP):
    if results_vg[win]:
        ax.plot(results_vg[win], label=f'T = {win}', color=color, lw=1.5)

ax.set_xlabel('Epoch')
ax.set_ylabel('‖∇W_hh‖₂  (log scale)')
ax.set_yscale('log')
ax.set_title('Vanishing Gradient vs longueur de fenêtre')
ax.legend(title='Taille fenêtre', fontsize=9)
ax.grid(alpha=0.3)

plt.tight_layout()
plt.savefig('../img/rnn_vanishing_gradient.png', dpi=150, bbox_inches='tight')
plt.show()
print('Sauvegardé dans img/rnn_vanishing_gradient.png')

## 11 — Conclusion et comparaison avec LSTM/GRU

| Modèle | Paramètres | Mémoire | Vanishing gradient |
|--------|-----------|---------|--------------------|
| **SimpleRNN** | ~1.5k | Courte (~3-5 pas) | **Oui** (critique) |
| **GRU** | ~40k | Longue | Non (porte de mise à jour) |
| **LSTM** | ~54k | Longue | Non (cellule de contexte) |

In [ ]:
print(f'=== Trace de la fenêtre glissante — {MODEL_NAME} ===')
print(f"{'Étape':<7} {'Jour':<12} {'Réel':>6} {'Prédit':>7} {'Err abs':>8} {'Err %':>7}   Rôle")
print('-' * 75)
for i, d in enumerate(DAY_LABELS):
    err_abs = abs(preds_real[i] - real_real[i])
    err_pct = 100 * err_abs / real_real[i]
    role = '← injecté dans la fenêtre suivante' if i < N_TEST - 1 else '← prédiction finale'
    print(f'  {i}      {d:<12} {real_real[i]:>6.0f} {preds_real[i]:>7.0f} {err_abs:>8.1f} {err_pct:>6.1f}%   {role}')

print(f'\nRésumé : MAE={mae:.1f} | MAPE={mape:.2f}% | RMSE={rmse:.1f} | R²={r2:.4f}')